In [3]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3373707393636829865, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18088110680284457903
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3298594438986026847
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17092955114208849490
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Archive:  /content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip
replace /content/fruits-360_dataset/fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [8]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['Test', 'papers', 'test-multiple_fruits', 'readme.md', 'LICENSE', 'Training']

In [9]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [10]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [11]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Mango               490
Onion Red           450
Cherry Rainier      738
Lemon               492
Walnut              735
Limes               490
Nut Forest          218
Cactus fruit        490
Pineapple Mini      493
Grapefruit White    492
Raspberry           490
Tomato Cherry Red   492
Apple Golden 2      492
Maracuja            490
Melon Piel de Sapo  738
Grape White 3       492
Quince              490
Onion White         438
Grape White 4       471
Grapefruit Pink     490
Potato White        450
Grape White         490
Carambula           490
Clementine          490
Cauliflower         702
Tamarillo           490
Eggplant            468
Apple Red Yellow 2  672
Banana              490
Lemon Meyer         490
Apple Red Yellow 1  492
Guava               490
Pear Abate          490
Strawberry Wedge    738
Plum                447
Apple Golden 1      492
Cherry 1            492
Tomato 4            479
Pepper Yellow       666
Cocos         

In [12]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
test_datagenerator = ImageDataGenerator(rescale= 1./255)

In [15]:
#Creating Batches
image_size = (256, 256)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [16]:
inception_v3_model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=image_size+(3,))
#Using pretrained
for layer in inception_v3_model.layers:
  layer.trainable=False
#Adding Layers at the top
top = inception_v3_model.output
top = GlobalAveragePooling2D(data_format='channels_last', name='avg_pool')(top)
top = Dropout(0.8)(top)


#Adding Classification layer with softmax activation function, (needed for categorical predictions)
predictor = Dense(120 ,activation='softmax', name='predictor', kernel_regularizer=keras.regularizers.l2(0.1))(top)
inception_v3_model = Model(inputs=inception_v3_model.input, outputs=predictor)

87916544/87910968 [==============================] - 3s 0us/step


In [17]:
inception_v3_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
#Compiling the model
inception_v3_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
inception_v3_model.save_weights('Initial weights.h5')

In [0]:
#Training and Validating the model: Round 1
wait = 1
epochs = 15
model_name='Inception v3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=wait)

In [19]:
#train only classification layer
#inception_v3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
#inception_v3_model.load_weights('Initial weights.h5')
trainingMdl = inception_v3_model.fit_generator(train_val_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/5
1891/1891 [==============================] - 1005s 532ms/step - loss: 5.4622 - accuracy: 0.1773 - val_loss: 5.4967 - val_accuracy: 0.1312
Epoch 2/5
1891/1891 [==============================] - 1006s 532ms/step - loss: 5.1467 - accuracy: 0.1904 - val_loss: 5.6417 - val_accuracy: 0.1313
Epoch 3/5
1891/1891 [==============================] - 1010s 534ms/step - loss: 5.1433 - accuracy: 0.1895 - val_loss: 5.6693 - val_accuracy: 0.0896
Epoch 4/5
1891/1891 [==============================] - 997s 527ms/step - loss: 5.1439 - accuracy: 0.1926 - val_loss: 5.9768 - val_accuracy: 0.1063
Epoch 5/5
1891/1891 [==============================] - 993s 525ms/step - loss: 5.1443 - accuracy: 0.1896 - val_loss: 5.2121 - val_accuracy: 0.0936


In [0]:
#train only classification layer
#inception_v3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
#inception_v3_model.load_weights('Initial weights.h5')
trainingMdl = inception_v3_model.fit_generator(train_val_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/5
1891/1891 [==============================] - 990s 523ms/step - loss: 1.2763 - accuracy: 0.7093 - val_loss: 4.9674 - val_accuracy: 0.2479
Epoch 2/5
1891/1891 [==============================] - 980s 518ms/step - loss: 0.4903 - accuracy: 0.8638 - val_loss: 3.6833 - val_accuracy: 0.2765
Epoch 3/5
1891/1891 [==============================] - 987s 522ms/step - loss: 0.4078 - accuracy: 0.8794 - val_loss: 4.4298 - val_accuracy: 0.2837
Epoch 4/5
1891/1891 [==============================] - 999s 529ms/step - loss: 0.3701 - accuracy: 0.8856 - val_loss: 8.4233 - val_accuracy: 0.2913
Epoch 5/5
1891/1891 [==============================] - 1007s 533ms/step - loss: 0.3661 - accuracy: 0.8867 - val_loss: 5.9708 - val_accuracy: 0.2716


In [25]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
from utils import plot_history
history_fig = plot_history(trainingMdl)
history_fig.show()

In [24]:
import importlib
importlib.reload(utils)

NameError: ignored